In [1]:
import os
import langchain
from langchain.llms import OpenAI

Load model

In [2]:
llm = OpenAI()
llm("Why did the chicken cross the road?")

'\n\nTo get to the other side.'

Load Documents

In [3]:
from langchain.document_loaders import TextLoader

In [25]:
loader = TextLoader("./birthday_ideas.txt")
doc = loader.load()
# print(doc)
# with open("./birthday_ideas.txt") as f:
#     birthday_doc = f.read()

Split text by tokens

In [45]:
from langchain.text_splitter import CharacterTextSplitter

In [61]:
text_splitter = CharacterTextSplitter(
chunk_size=5,
chunk_overlap=0,
length_function=len
)

In [62]:
texts = text_splitter.split_documents(doc)

In [63]:
texts

[Document(page_content="Birthdays in the family: \n- Zarah's birthday is in February \n- Max's birthday is in April \n- James' birthday is in May \n- Mom's birthday is in July \n- Dad's birthday is in October \n- Zyde's birthday is in December", metadata={'source': './birthday_ideas.txt'})]

Embed chunks 

In [22]:
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings

In [23]:
embeddings = HuggingFaceEmbeddings()

In [65]:
embedding_result = embeddings.embed_documents(texts)

AttributeError: 'Document' object has no attribute 'replace'